In [ ]:
import os
from ultralytics import YOLO
import cv2
import xgboost as xgb
import pandas as pd

def detect_shoplifting(video_path):
    model_yolo = YOLO(r"D:\Final year project\yolo version project\best.pt")
    model = xgb.Booster()
    model.load_model(r"D:\Final year project\yolo version project\model_weights.json")

    cap = cv2.VideoCapture(video_path)

    print('Total Frame', cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
    
    # Generate a unique output path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_path = fr"D:\Final year project\yolo version project\output video\{video_name}_output.mp4"
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_tot = 0

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            results = model_yolo(frame, verbose=False)

            # Visualize the results on the frame
            annotated_frame = results[0].plot(boxes=False)

            for r in results:
                bound_box = r.boxes.xyxy
                conf = r.boxes.conf.tolist()
                keypoints = r.keypoints.xyn.tolist()

                print(f'Frame {frame_tot}: Detected {len(bound_box)} bounding boxes')

                for index, box in enumerate(bound_box):
                    if conf[index] > 0.75:
                        x1, y1, x2, y2 = box.tolist()
                        data = {}

                        # Initialize the x and y lists for each possible key
                        for j in range(len(keypoints[index])):
                            data[f'x{j}'] = keypoints[index][j][0]
                            data[f'y{j}'] = keypoints[index][j][1]

                        # print(f'Bounding Box {index}: {data}')
                        df = pd.DataFrame(data, index=[0])
                        dmatrix = xgb.DMatrix(df)
                        cut = model.predict(dmatrix)
                        binary_predictions = (cut > 0.5).astype(int)
                        print(f'Prediction: {binary_predictions}')

                        if binary_predictions == 0:
                            conf_text = f'Suspicious ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 7, 58), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 7, 58), 2)
                        if binary_predictions == 1:
                            conf_text = f'Normal ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (57, 255, 20), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (57, 255, 20), 2)


            cv2.imshow('Frame', annotated_frame)

            out.write(annotated_frame)
            frame_tot += 1
            print('Processed Frame:', frame_tot)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage:
video_path = r"D:\Final year project\yolo version project\Test\istockphoto-668181452-640_adpp_is.mp4"
detect_shoplifting(video_path)


Total Frame 401.0
Frame 0: Detected 2 bounding boxes
Prediction: [0]
Prediction: [0]
Processed Frame: 1
Frame 1: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 2
Frame 2: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 3
Frame 3: Detected 2 bounding boxes
Prediction: [1]
Prediction: [1]
Processed Frame: 4
Frame 4: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 5
Frame 5: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 6
Frame 6: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 7
Frame 7: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 8
Frame 8: Detected 2 bounding boxes
Prediction: [0]
Prediction: [0]
Processed Frame: 9
Frame 9: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 10
Frame 10: Detected 2 bounding boxes
Prediction: [0]
Prediction: [1]
Processed Frame: 11
Frame 11: Detected 2 bounding box

In [2]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from ultralytics import YOLO
import cv2
import xgboost as xgb
import pandas as pd

def send_email_alert(frame_number, confidence, video_path, frame):
    try:
        # SMTP Configuration
        smtp_server = 'smtp.gmail.com'
        smtp_port = 587
        sender_email = "suspicious616@gmail.com"
        sender_password = "fpvk xpog jkfq gjpp"  # Replace with actual app password
        recipient_emails = ['bhuvanm411354@gmail.com', 'suspicious616@gmail.com']
        
        # Email Content
        subject = "🚨 Suspicious Activity Detected!"
        body = (f"Security Alert Notification\n\n"
                f"⚠ Suspicious activity detected in surveillance video\n"
                f"• Video: {os.path.basename(video_path)}\n"
                f"• Frame: {frame_number}\n"
                f"• Confidence: {confidence:.2%}\n\n"
                f"Please review the attached frame immediately.")
        
        # Create MIME Message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = ", ".join(recipient_emails)
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))
        
        # Attach Frame Image
        _, img_encoded = cv2.imencode('.jpg', frame)
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(img_encoded.tobytes())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="alert_frame_{frame_number}.jpg"')
        msg.attach(part)
        
        # Send Email
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(msg)
        
        print(f"Alert email sent for frame {frame_number} (Confidence: {confidence:.2%})")
        
    except Exception as e:
        print(f"Failed to send email alert: {str(e)}")

def detect_shoplifting(video_path):
    model_yolo = YOLO(r"D:\Final year project\yolo version project\best.pt")
    model = xgb.Booster()
    model.load_model(r"D:\Final year project\yolo version project\model_weights.json")

    cap = cv2.VideoCapture(video_path)
    print('Total Frame', cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_path = fr"D:\Final year project\yolo version project\output video\{video_name}_output.mp4"
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_tot = 0

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            results = model_yolo(frame, verbose=False)
            annotated_frame = results[0].plot(boxes=False)

            for r in results:
                bound_box = r.boxes.xyxy
                conf = r.boxes.conf.tolist()
                keypoints = r.keypoints.xyn.tolist()

                print(f'Frame {frame_tot}: Detected {len(bound_box)} bounding boxes')

                for index, box in enumerate(bound_box):
                    if conf[index] > 0.75:
                        x1, y1, x2, y2 = box.tolist()
                        data = {}

                        for j in range(len(keypoints[index])):
                            data[f'x{j}'] = keypoints[index][j][0]
                            data[f'y{j}'] = keypoints[index][j][1]

                        df = pd.DataFrame(data, index=[0])
                        dmatrix = xgb.DMatrix(df)
                        cut = model.predict(dmatrix)
                        binary_predictions = (cut > 0.5).astype(int)
                        print(f'Prediction: {binary_predictions}')

                        if binary_predictions == 0:  # Suspicious activity
                            conf_text = f'Suspicious ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 7, 58), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 7, 58), 2)
                            
                            if conf[index] >= 0.85:  # Only send email for high-confidence detections
                                send_email_alert(frame_tot, conf[index], video_path, annotated_frame)
                                
                        elif binary_predictions == 1:  # Normal activity
                            conf_text = f'Normal ({conf[index]:.2f})'
                            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (57, 255, 20), 2)
                            cv2.putText(annotated_frame, conf_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 1.0, (57, 255, 20), 2)

            cv2.imshow('Frame', annotated_frame)
            out.write(annotated_frame)
            frame_tot += 1
            print('Processed Frame:', frame_tot)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage:
video_path = r"D:\Final year project\yolo version project\10480748-uhd_4096_2160_25fps.mp4"
detect_shoplifting(video_path)

Total Frame 1072.0
Frame 0: Detected 2 bounding boxes
Processed Frame: 1
Frame 1: Detected 2 bounding boxes
Processed Frame: 2
Frame 2: Detected 2 bounding boxes
Processed Frame: 3
Frame 3: Detected 2 bounding boxes
Processed Frame: 4
Frame 4: Detected 2 bounding boxes
Processed Frame: 5
Frame 5: Detected 2 bounding boxes
Processed Frame: 6
Frame 6: Detected 2 bounding boxes
Processed Frame: 7
Frame 7: Detected 2 bounding boxes
Processed Frame: 8
Frame 8: Detected 2 bounding boxes
Processed Frame: 9
Frame 9: Detected 2 bounding boxes
Processed Frame: 10
Frame 10: Detected 2 bounding boxes
Processed Frame: 11
Frame 11: Detected 2 bounding boxes
Processed Frame: 12
Frame 12: Detected 2 bounding boxes
Processed Frame: 13
Frame 13: Detected 2 bounding boxes
Processed Frame: 14
Frame 14: Detected 2 bounding boxes
Processed Frame: 15
Frame 15: Detected 2 bounding boxes
Processed Frame: 16
Frame 16: Detected 2 bounding boxes
Processed Frame: 17
Frame 17: Detected 2 bounding boxes
Processed Fr

KeyboardInterrupt: 